In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU, Input, Normalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Model
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view

## Loading & Processing Input Data after extraction from the .mlx file script

In [3]:
df_train = pd.read_csv('train.txt')
df_val = pd.read_csv('val.txt')
df_test = pd.read_csv('test.txt')

In [4]:
# train
df_train['X_real'] = df_train['Var1_2'].str.extract(r'([+-]?\d+\.\d+(?:[eE][+-]?\d+)?)').astype(float)
df_train['X_img'] = df_train['Var1_2'].str.extract(r'([+-]?\d+\.\d+(?:[eE][+-]?\d+)?)i').astype(float)
df_train['y_real'] = df_train['Var1_1'].str.extract(r'([+-]?\d+\.\d+(?:[eE][+-]?\d+)?)').astype(float)
df_train['y_img'] = df_train['Var1_1'].str.extract(r'([+-]?\d+\.\d+(?:[eE][+-]?\d+)?)i').astype(float)

# test
df_test['X_real'] = df_test['Var1_2'].str.extract(r'([+-]?\d+\.\d+(?:[eE][+-]?\d+)?)').astype(float)
df_test['X_img'] = df_test['Var1_2'].str.extract(r'([+-]?\d+\.\d+(?:[eE][+-]?\d+)?)i').astype(float)
df_test['y_real'] = df_test['Var1_1'].str.extract(r'([+-]?\d+\.\d+(?:[eE][+-]?\d+)?)').astype(float)
df_test['y_img'] = df_test['Var1_1'].str.extract(r'([+-]?\d+\.\d+(?:[eE][+-]?\d+)?)i').astype(float)

# val
df_val['X_real'] = df_val['Var1_2'].str.extract(r'([+-]?\d+\.\d+(?:[eE][+-]?\d+)?)').astype(float)
df_val['X_img'] = df_val['Var1_2'].str.extract(r'([+-]?\d+\.\d+(?:[eE][+-]?\d+)?)i').astype(float)
df_val['y_real'] = df_val['Var1_1'].str.extract(r'([+-]?\d+\.\d+(?:[eE][+-]?\d+)?)').astype(float)
df_val['y_img'] = df_val['Var1_1'].str.extract(r'([+-]?\d+\.\d+(?:[eE][+-]?\d+)?)i').astype(float)

In [5]:
X_train = df_train[['X_real','X_img']].to_numpy()
y_train = df_train[['y_real','y_img']].to_numpy()

X_test = df_test[['X_real','X_img']].to_numpy()
y_test = df_test[['y_real','y_img']].to_numpy()

X_val = df_val[['X_real','X_img']].to_numpy()
y_val = df_val[['y_real','y_img']].to_numpy()

In [7]:
y_val

array([[ 0.00150483,  0.00166845],
       [ 0.00151624,  0.00048847],
       [ 0.00165905, -0.00093272],
       ...,
       [ 0.00218926,  0.00537777],
       [ 0.00462811,  0.00423432],
       [ 0.00672882,  0.00265553]])

In [8]:
def preprocess(arr, n_previous = 4):
    array_with_l1_norm  = np.hstack((arr, np.sum(np.abs(arr),axis=1)[:, np.newaxis]))

    padding = np.zeros((n_previous, array_with_l1_norm.shape[1]))
    padded_array = np.vstack((padding, array_with_l1_norm ))

    windows = sliding_window_view(padded_array, window_shape=(n_previous + 1, array_with_l1_norm.shape[1]))
    windows = windows.reshape(windows.shape[0], -1)
    
    return windows

In [9]:
X_train = preprocess(X_train)
X_test = preprocess(X_test)
X_val = preprocess(X_val)


In [8]:
X_train.shape

(131520, 15)

In [8]:
# from sklearn import preprocessing

# x_min_max_scaler = preprocessing.MinMaxScaler()
# y_min_max_scaler = preprocessing.MinMaxScaler()

# X_train = x_min_max_scaler.fit_transform(X_train)
# y_train = y_min_max_scaler.fit_transform(y_train)


In [9]:
# X_val = x_min_max_scaler.transform(X_val)
# y_val = y_min_max_scaler.transform(y_val)

# X_test = x_min_max_scaler.transform(X_test)
# y_test = y_min_max_scaler.transform(y_test)

In [10]:
# # Build the model using Sequential API
# model = Sequential([
#     Dense(30),
#     LeakyReLU(alpha=0.01),
#     Dense(30),
#     LeakyReLU(alpha=0.01),
#     Dense(30),
#     LeakyReLU(alpha=0.01),
#     Dense(2)
# ])

# # Compile the model
# model.compile(optimizer=Adam(learning_rate=4e-4), 
#               loss='mean_squared_error', 
#               metrics=['mean_squared_error'])

# # Define training parameters
# maxEpochs = 200
# miniBatchSize = 1024
# iterPerEpoch = len(X_train) // miniBatchSize
# validation_freq = 2 * iterPerEpoch

# # Callbacks for learning rate adjustment, early stopping, and model checkpoint
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.95, patience=5, verbose=1, mode='auto')
# # early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

# # Train the model
# history = model.fit(
#     X_train, y_train,
#     epochs=maxEpochs,
#     batch_size=miniBatchSize,
#     validation_data=(X_val, y_val),
#     callbacks=[reduce_lr],
#     shuffle=True,
#     verbose=1
# )

In [9]:
def helperNMSE(y_true, y_pred):
    
    diff = y_pred - y_true
    mse = tf.reduce_mean(tf.norm(diff,axis=1)**2) # NOTE THIS IS NOT A GOOD PRACICE AS TF.NORM ALREADY GETS THE SQUARED ERROR THEN SQRT IT
    factor = tf.reduce_mean(tf.norm(y_true,axis=1)**2)
    # nmse = 10 * tf.math.log(mse / factor) / tf.math.log(tf.constant(10,dtype=tf.float32))
    
    return mse

def custom_loss(y_true, y_pred):
    # Compute the L1 norms
    norm_true = tf.reduce_sum(tf.abs(y_true), axis=1)
    norm_pred = tf.reduce_sum(tf.abs(y_pred), axis=1)
    
    # Reshape to be compatible with the helperNMSE function
    norm_true = tf.reshape(norm_true, (-1, 1))
    norm_pred = tf.reshape(norm_pred, (-1, 1))
    
    # Compute the NMSE
    loss = helperNMSE(norm_true, norm_pred)
    return loss

In [12]:
# a  = tf.convert_to_tensor([[2,1],[5,6]],dtype=tf.float32)
# b = tf.convert_to_tensor([[2,5],[2,4]],dtype=tf.float32)


In [13]:
# helperNMSE(a,b)

In [10]:
# defining layers
input_layer = Input(shape=(X_train.shape[1],))
dense_layer_1 = Dense(units = 30, activation = LeakyReLU(alpha=0.01))(input_layer) 
dense_layer_2 = Dense(units = 24, activation = LeakyReLU(alpha=0.01))(dense_layer_1)
dense_layer_3 = Dense(units = 19, activation = LeakyReLU(alpha=0.01))(dense_layer_2)




#Y1 output
y_output = Dense(units = 2, activation = "linear", name = "y1_output")(dense_layer_3)

#Y2 output

#Define the model with the input layer and a list of outputs
model = Model(inputs = input_layer, outputs = [y_output])


reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.95, patience=5, verbose=1, mode='auto')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')
checkpoint_callback = ModelCheckpoint(
    filepath='bestmodel.keras',
    monitor='val_loss',       # Metric to monitor
    save_best_only=True,      # Only save the model if it is the best
    verbose=0                 # Verbosity mode
)



#specify the optimizer and compile with the loss function for both outputs
optimizer = tf.keras.optimizers.Adam(learning_rate=4e-4)

model.compile(optimizer = optimizer,
              loss = 'mse',
             )


c:\Users\abood\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [11]:
# model.load_weights('tf_model_33_5dB_nprev_4.h5')

In [11]:
# Define training parameters
maxEpochs = 700 # val_loss ~ 3.5e-8, NMSE=-33.416, n_prev = 4. [increase to 700 if didn't achieve this]
miniBatchSize = 1024
iterPerEpoch = len(X_train) // miniBatchSize
validation_freq = 2 * iterPerEpoch


# history = model.fit(X_train, (y_train.y_real, y_train.y_img), epochs = 200, batch_size = 10,
#                     validation_data = (X_val, (y_val.y_real, y_val.y_img)))


# Train the model
history = model.fit(
    X_train, y_train,
    epochs=maxEpochs,
    batch_size=miniBatchSize,
    callbacks=[checkpoint_callback,reduce_lr],
    validation_data = (X_val, y_val),
    shuffle=True,
)

Epoch 1/700
129/129 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 2.0862e-05 - val_loss: 1.0797e-06 - learning_rate: 4.0000e-04
Epoch 2/700
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8.4882e-07 - val_loss: 5.2731e-07 - learning_rate: 4.0000e-04
Epoch 3/700
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.1766e-07 - val_loss: 4.1023e-07 - learning_rate: 4.0000e-04
Epoch 4/700
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3.9616e-07 - val_loss: 3.6466e-07 - learning_rate: 4.0000e-04
Epoch 5/700
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3.3540e-07 - val_loss: 2.8090e-07 - learning_rate: 4.0000e-04
Epoch 6/700
 73/129 ━━━━━━━━━━━━━━━━━━━━ 0s 700us/step - loss: 2.9063e-07
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.00037999999040039256.
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.8394e-07 - val_loss: 2.7088e-07 - learning_rate: 4.0000e-04
Epoch 7/700
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.3817e-07 - val_loss: 2.0127e-07 - learning_rate: 3.8000e-04


In [14]:
# model.load_weights('tf_model_33_5dB_nprev_4.h5')

In [12]:
model.save('tf_model_single_y.h5')

In [18]:
# model.predict(X_train)

In [19]:
# y_pred = tf.reshape(model.predict(X_test),(131520,2))

In [20]:
# helperNMSE(tf.convert_to_tensor(y_test,dtype=tf.float32),y_pred)

In [ ]:
(pd.DataFrame(history.history['loss'])).plot()

In [15]:
tf_output = model.predict(preprocess(y_test))


4110/4110 ━━━━━━━━━━━━━━━━━━━━ 2s 443us/step


In [16]:
pd.DataFrame(tf_output).to_csv('tmp.csv')

In [17]:
arr = []

for i in range(len(tf_output)):
    arr.append(f'{tf_output[i][0]} + {tf_output[i][1]}i')


In [19]:
pd.Series(arr).to_csv('tf_model_output_test.csv',index=False,header=False)


In [20]:
import numpy as np
import scipy.io

# Function to parse complex numbers from the given format
def parse_complex_number(s):
    real, imag = s.split(' + ')
    real = float(real)
    imag = float(imag.replace('i', ''))
    return np.complex64(real + 1j * imag)

# Read the CSV file
filename = './tf_model_output_test.csv'  # Replace with your CSV filename
with open(filename, 'r') as file:
    lines = file.readlines()

# Parse the lines into complex numbers
complex_numbers = np.array([parse_complex_number(line.strip()) for line in lines], dtype=np.complex64)

# Save the array to a .mat file
output_filename = 'complex_data.mat'
scipy.io.savemat(output_filename, {'complex_data': complex_numbers})


## Exporting to ONNX For the use in fpgaConvnet

In [15]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 30)        │        480 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 24)        │        744 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 19)        │        475 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ y1_output (Dense)   │ (None, 1)         │         20 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ y2_output (Dense)   │ (None, 1)         │         20 │ dense_2[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,739 (6.79 KB)

 Trainable params: 1,739 (6.79 KB)

 Non-trainable params: 0 (0.00 B)

In [28]:
import tf2onnx
import onnx


input_signature =[tf.TensorSpec([None,15], tf.float32, name='x')]

# Use from_function for tf functions

onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature)

onnx.save(onnx_model, "tf_model_33_5dB_nprev_4.onnx")
